# ResNet

ResNet 模型由 Kaiming He、Xiangyu Zhang、Shaoqing Ren 和 Jian Sun 在 [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) 一文中提出。

<div class="alert alert-info">
**摘要**：深度神经网络更难训练。我们提出了一种残差学习框架，以简化比以前使用的网络深度大得多的网络的训练。我们明确地将层重新表述为参照层输入学习残差函数，而非学习无参考的函数。我们提供了全面的经验证据，表明这些残差网络更易于优化，并且可以从大幅增加的深度中获得准确性。在 ImageNet 数据集上，我们评估了深度高达 152 层的残差网络 —— 比 VGG 网络深 8 倍，但复杂度仍然较低。这些残差网络的集成在 ImageNet 测试集上实现了 3.57% 的错误率。这一结果在 2015 年 ILSVRC 分类任务中获得了第一名。我们还展示了在具有 100 层和 1000 层的 CIFAR-10 上的分析。表征网络的深度对于许多视觉识别任务至关重要。仅仅由于我们对于深度的提升，我们在 COCO 目标检测数据集上获得了 28% 的相对改进。深度残差网络是我们提交给 ILSVRC 和 COCO 2015 竞赛的基础，在那里我们也在 ImageNet 检测、ImageNet 定位、COCO 检测和 COCO 分割任务中获得了第一名。
</div>

神经网络随着层数增加而被认为更加有效，因为增加层数可以提升模型的表现。随着网络的加深，提取的特征可以得到进一步丰富，例如 VGG16 和 VGG19。于是产生了一个疑问：“网络的学习是否也像简单地堆叠更多的层一样容易？” 为了解决这个问题中的一个障碍——梯度消失问题，采用了归一化的初始化方式和中间归一化层。

然而，一个新的问题出现了：退化问题。随着神经网络变得更深，准确度趋于饱和并快速退化。一项对比 shadow 和 deep plain 网络的实验表明，深层模型在训练和测试中表现出更高的错误率，表明在有效训练深层架构方面存在根本性的挑战。这种退化不是由于过拟合，而是因为网络变深时训练误差增加。增加的层未能逼近身份函数。

ResNet的残差连接释放了极深度网络的潜力，相较于先前架构显著提升了准确性。


<div class="wy-nav-content-img">
    <img src="assets/ResNet_resnet_archs.png" alt="34 层的ResNet架构与 VGG19 以及线性连接的结构之间的对比">
    <p>图1: 34 层的ResNet架构与 VGG19 以及线性连接的结构之间的对比</p>
</div>



## ResNet 架构

ResNet的构建块被设计为身份函数，在保留输入信息的同时进行学习。这种方法确保了高效的权重优化，并防止了网络加深时的退化。

ResNet的构建模块如图所示，来源：ResNet论文。

<div class="wy-nav-content-img">
    <img src="assets/ResNet_resnet_building_block.png" width=400px alt="ResNet 架构中的残差块">
    <p>图2: ResNet 架构中的残差块</p>
</div>



快捷连接执行身份映射，其输出会加到堆叠层的输出上。身份快捷连接既不增加额外的参数也不增加计算复杂度，这些连接绕过层级，为信息流动创造了直接路径，使得神经网络能够学习残差函数 (F)。

我们可以将ResNet网络总结为 -> 平面网络 + 快捷连接！

对于操作 $F(x) + x$，$F(x)$ 和 $x$ 应具有相同的维度。 ResNet采用了两种技术来实现这一点：

* 零填充快捷连接：添加全为零的通道来保持维度，同时不引入额外需要学习的参数。
* 投影快捷连接：使用1x1卷积在必要时调整维度，包含一些可学习的额外参数。

在更深的ResNet架构中，例如ResNet 50、101 和 152，采用了一种专门的“瓶颈构建块”来管理参数复杂性，保持效率，同时允许更深度的学习。



## ResNet 的从零实现

In [1]:
import torch
from torch import nn
from torch import Tensor

### ResNetConvLayer

ResNetConvLayer 是 ResNet 中最基础的 Layer 结构，它是由一个 Conv 层，一个 Norm 层和一个非线性激活层组成的。

In [2]:
class ResNetConvLayer(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int = 3,
        stride: int = 1,
        activation: nn.Module = nn.ReLU,
    ):
        super().__init__()
        self.convolution = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=kernel_size // 2,
            bias=False,
        )
        self.normalization = nn.BatchNorm2d(out_channels)
        self.activation = activation() if activation is not None else nn.Identity()

    def forward(self, input: Tensor) -> Tensor:
        hidden_state = self.convolution(input)
        hidden_state = self.normalization(hidden_state)
        hidden_state = self.activation(hidden_state)
        return hidden_state

### ResNetBottleNeckLayer

在ResNet中，BottleNeck结构主要用于ResNet-50及其以上的深层网络（如ResNet-101和ResNet-152）。这种设计通过使用3层卷积而不是基本的2层卷积来构建残差块，有效地减少了计算量和参数数量。具体来说，BottleNeck结构通常由以下三个卷积层组成：

* `1x1` 卷积（降维）：减少输入通道的维度，从而降低计算量。
* `3x3` 卷积：在降维后的空间进行卷积操作，提取特征。
* `1x1` 卷积（升维）：将通道维度恢复到原始的维度。

通过这种设计，ResNet的BottleNeck结构能够在保持网络深度和表现力的同时，减少计算需求，从而更高效地应用于深层网络。

ResNetBottleNeckLayer 支持通过第一个 1x1 Conv 的 stride 来降低特征图的尺寸，一般来说 stride=2 将特征图的尺寸降低一倍，这个时候，ShortCut 部分就需要匹配一个 1x1 stride=2 的 Conv，使得最后的 FeatureMap 的维度一致。

<div class="wy-nav-content-img">
    <img src="./assets/ResNet_BottleNeckLayer.drawio.svg" width=600px alt="原始的 BasicLayer 和 BottleNeckLayer 结构的对比">
    <p>图3: 原始的 BasicLayer 和 BottleNeckLayer 结构的对比</p>
</div>

In [3]:
class ResNetBottleNeckLayer(nn.Module):
    """
    A classic ResNet's bottleneck layer composed by three `3x3` convolutions.

    The first `1x1` convolution reduces the input by a factor of `reduction` in order to make the second `3x3`
    convolution faster. The last `1x1` convolution remaps the reduced features to `out_channels`. If
    `downsample_in_bottleneck` is true, downsample will be in the first layer instead of the second layer.
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        stride: int = 1,
        activation: nn.Module = nn.ReLU,
        reduction: int = 4,
        downsample_in_bottleneck: bool = False,
    ):
        super().__init__()
        should_apply_shortcut = in_channels != out_channels or stride != 1
        reduces_channels = out_channels // reduction
        self.shortcut = (
            nn.Sequential(
                nn.Conv2d(
                    in_channels, out_channels, kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(out_channels),
            )
            if should_apply_shortcut
            else nn.Identity()
        )
        self.layer = nn.Sequential(
            ResNetConvLayer(
                in_channels,
                reduces_channels,
                kernel_size=1,
                stride=stride if downsample_in_bottleneck else 1,
            ),
            ResNetConvLayer(
                reduces_channels,
                reduces_channels,
                stride=stride if not downsample_in_bottleneck else 1,
            ),
            ResNetConvLayer(
                reduces_channels, out_channels, kernel_size=1, activation=None
            ),
        )
        self.activation = activation() if activation is not None else nn.Identity()

    def forward(self, hidden_state):
        residual = hidden_state
        hidden_state = self.layer(hidden_state)
        residual = self.shortcut(residual)
        hidden_state += residual
        hidden_state = self.activation(hidden_state)
        return hidden_state


class ResNetBasicLayer(nn.Module):
    """
    A classic ResNet's residual layer composed by two `3x3` convolutions.
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        stride: int = 1,
        activation: nn.Module = nn.ReLU,
    ):
        super().__init__()
        should_apply_shortcut = in_channels != out_channels or stride != 1
        self.shortcut = (
            nn.Sequential(
                nn.Conv2d(
                    in_channels, out_channels, kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(out_channels),
            )
            if should_apply_shortcut
            else nn.Identity()
        )
        self.layer = nn.Sequential(
            ResNetConvLayer(in_channels, out_channels, stride=stride),
            ResNetConvLayer(out_channels, out_channels, activation=None),
        )
        self.activation = activation() if activation is not None else nn.Identity()

    def forward(self, hidden_state):
        residual = hidden_state
        hidden_state = self.layer(hidden_state)
        residual = self.shortcut(residual)
        hidden_state += residual
        hidden_state = self.activation(hidden_state)
        return hidden_state

### ResNetStage

ReNetStage 是对 ResNet 模型结构中一个大的处理阶段的包装。每个 Stage 都是由若干个（由 depth 控制）ResNetBottleNeckLayer 或 ResNetBasicLayer 构成的。除了第一个 Stage外，其余的每个 Stage 的第一个 Layer 往往会将整个 FeatureMap 的尺寸降低一倍。

In [4]:
class ResNetStage(nn.Module):
    """
    A ResNet stage composed by stacked layers.
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        stride: int = 2,
        depth: int = 2,
        layer_type="bottleneck",
        downsample_in_bottleneck=False,
        hidden_act: nn.Module = nn.ReLU,
    ):
        super().__init__()

        layer = (
            ResNetBottleNeckLayer if layer_type == "bottleneck" else ResNetBasicLayer
        )

        if layer_type == "bottleneck":
            first_layer = layer(
                in_channels,
                out_channels,
                stride=stride,
                activation=hidden_act,
                downsample_in_bottleneck=downsample_in_bottleneck,
            )
        else:
            first_layer = layer(
                in_channels, out_channels, stride=stride, activation=hidden_act
            )
        self.layers = nn.Sequential(
            first_layer,
            *[
                layer(out_channels, out_channels, activation=hidden_act)
                for _ in range(depth - 1)
            ]
        )

    def forward(self, input: Tensor) -> Tensor:
        hidden_state = input
        for layer in self.layers:
            hidden_state = layer(hidden_state)
        return hidden_state

### 组装成 ResNet 模型

<div class="wy-nav-content-img">
    <img src="assets/ResNet_model_variants.png" alt="原始的 BasicLayer 和 BottleNeckLayer 结构的对比">
    <p>图4: ResNet的详细架构图</p>
</div>

ResNet 模型的整个可以分为以下几个部分：

1. Stem 部分是由一个 7x7，步幅=2 的卷积构成。
2. 在 Stem 后会接一个 2x2 的 MaxPooling 将整个特征图的尺寸再减小一半
3. Stage1 ~ Stage 4 四个 ResNetStage 组成，最后一个 Stage 输出的特征图大小为 7x7
4. `AdaptiveAvgPool2d` 将整个特征图的维度压缩成 1x1 的尺寸
5. 一个全连接分类头。



In [5]:
class ResNetModel(nn.Module):
    def __init__(
        self, num_channels, embedding_size, depths, hidden_sizes, num_classes=1000
    ):
        super().__init__()
        self.embeder = ResNetConvLayer(
            num_channels, embedding_size, kernel_size=7, stride=2
        )
        self.embed_pooler = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.stages = nn.ModuleList()
        # 先添加第一个 stage
        self.stages.append(
            ResNetStage(
                in_channels=embedding_size,
                out_channels=hidden_sizes[0],
                stride=1,
                depth=depths[0],
            )
        )
        in_out_channels = zip(hidden_sizes[:-1], hidden_sizes[1:])
        for (in_chans, out_chans), depth in zip(in_out_channels, depths[1:]):
            self.stages.append(ResNetStage(in_chans, out_chans, stride=2, depth=depth))

        self.pooler = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(), nn.Linear(hidden_sizes[-1], num_classes)
        )

    def forward(self, x):
        embedding_output = self.embed_pooler(self.embeder(x))

        encoder_output = embedding_output
        for stage in self.stages:
            encoder_output = stage(encoder_output)

        pooled_output = self.pooler(encoder_output)

        classifer_output = self.classifier(pooled_output)
        return classifer_output

In [6]:
depths = [3, 4, 6, 3]
hidden_sizes = [256, 512, 1024, 2048]

model = ResNetModel(
    num_channels=3, embedding_size=64, depths=depths, hidden_sizes=hidden_sizes
)

In [7]:
out = model(torch.randn(1, 3, 224, 224))
out.shape

torch.Size([1, 1000])

## 使用 Transformers 库

In [8]:
from transformers import ResNetForImageClassification, ResNetConfig

model_cfg = ResNetConfig()
model_cfg.num_labels = 1024
model = ResNetForImageClassification(model_cfg)

x = torch.randn(1, 3, 224, 224)

output = model(x)
print(output.logits.shape)

torch.Size([1, 1024])


In [9]:
from transformers import AutoImageProcessor
from PIL import Image
import requests

image = Image.open("assets/cats_image.jpeg")

image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

inputs = image_processor(image, return_tensors="pt")

with torch.inference_mode():
    logits = model(**inputs).logits

predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

tiger cat
